<a href="https://colab.research.google.com/github/zlximon/Tensorflow_DL/blob/master/NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#downloading helper functions 
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py


--2022-07-12 15:19:55--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-07-12 15:19:55 (103 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [2]:
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys



#get text dataset

In [3]:
!wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"

# Unzip data
unzip_data("nlp_getting_started.zip")

--2022-07-12 15:19:58--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.191.128, 173.194.192.128, 142.250.125.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.191.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.004s  

2022-07-12 15:19:58 (154 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



  #visualizing text dataset

In [4]:
import pandas as pd 

In [5]:
train_df=pd.read_csv("train.csv")
test_df=pd.read_csv("test.csv")
train_df.head()
# 1 is for disaster 

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:
train_df_shuffled=train_df.sample(frac=1,random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [7]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [8]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [9]:
len(train_df),len(test_df)

(7613, 3263)

In [10]:
import random
random_index = random.randint(0, len(train_df)-5) # create random indexes not higher than the total number of samples
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
  print(f"Text:\n{text}\n")
  print("---\n")

Target: 0 (not real disaster)
Text:
Coursing* '@WEYREY_gidi: Now they are causing Di Maria.. LOL'

---

Target: 1 (real disaster)
Text:
#floods #ukfloods - British trekkers rescued amid flash floods in Himalayas: A group of British tr... http://t.co/nsfzkfgZnj - #flooding

---

Target: 0 (not real disaster)
Text:
Detonation into the realistic assets entering india: koZ http://t.co/9ZRQMd8nGZ

---

Target: 0 (not real disaster)
Text:
Black Eye 9: A space battle occurred at Star O784 involving 3 fleets totaling 3942 ships with 14 destroyed

---

Target: 0 (not real disaster)
Text:
Just bought another @meinlcymbals 18' medium crash!! Hey @meinlcymbals what about an endorsement! Starting to get expensive!

---



#split data into training and validation  sets

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
train_sentences,val_sentences,train_labels,val_labels=train_test_split(train_df_shuffled["text"].to_numpy(),train_df_shuffled["target"],test_size=0.1,
                                                                       random_state=42)

In [13]:
len(train_sentences),len(val_sentences),len(train_labels),len(val_labels)

(6851, 762, 6851, 762)

In [14]:
train_sentences[:10],train_labels[:10 ]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object), 5921    0


Embedding : create a matrix of features vector for each token 

#text vectorization ( tokenization)

In [15]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [16]:
text_vectorizer=TextVectorization(max_tokens=None# how many word in vocalubary  and None means there is no cap 
                                  ,standardize="lower_and_strip_punctuation",# turn all words to lower case and strip 
                                  split="whitespace",ngrams=None,output_mode="int",output_sequence_length=None,# how long do we want sequences be 
                                 )# pad_to_max_tokens=True)#add zeros into end of short lenght 

In [17]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# Use the default TextVectorization parameters
text_vectorizer = TextVectorization(max_tokens=None, # how many words in the vocabulary (automatically add <OOV>)
                                    standardize="lower_and_strip_punctuation",
                                    split="whitespace",
                                    ngrams=None, # create groups of n-words?
                                    output_mode="int", # how to map tokens to numbers
                                    output_sequence_length=None # how long do you want your sequences to be?
                                    )

In [18]:
#find the average numbers of tokens
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))


15

In [19]:
# Setup text vectorization with custom variables
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 15 # max length our sequences will be (e.g. how many words from a Tweet does our model see?)

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [20]:
# fit the text vectorization to the training text
text_vectorizer.adapt(train_sentences)


In [21]:
#create a sample 
sample="There is a flood in my street"
text_vectorizer([sample])


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 74,   9,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [22]:
#choose a random sentence from training dataset
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
        \n\nVectorized version:")
text_vectorizer([random_sentence])

Original text:
Currency transgress before payday-prison ward sinking-fund payment unsecured loan: jBUmZQpK        

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[   1, 7406,  239, 9899, 6937, 8488, 9897, 7178, 2891,    1,    0,
           0,    0,    0,    0]])>

In [23]:
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5] # the most common words in the vocab
bottom_5_words = words_in_vocab[-5:] # the least common words in the vocab
print(f"Most common words in vocab: {top_5_words}")
print(f"Least common words in vocab: {bottom_5_words}")

Most common words in vocab: ['', '[UNK]', 'the', 'a', 'in']
Least common words in vocab: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


#creating Embedding layer 

Embedding : turns positives integers into dense vectors of fixed size
parameters for our embedding

1-input_dim - The size of the vocabulary

2-output_dim - The size of the output embedding vector, for example, a value of 100 outputs a feature vector of size 100 for each word.

3-embeddings_initializer - How to initialize the embeddings matrix, default is "uniform" which randomly initalizes embedding matrix with uniform distribution. This can be changed for using pre-learned embeddings.

4- input_length - Length of sequences being passed to embedding layer.


In [24]:
from tensorflow.keras import layers 
embedding =layers.Embedding(input_dim=max_vocab_length,output_dim=128,input_length=max_length)
embedding

In [25]:
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
      \n\nEmbedded version:")

# Embed the random sentence (turn it into numerical representation)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
Me trying to look cute wen crush is passing by ... http://t.co/Z87zMi3Ozs      

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.03648127,  0.03756154,  0.00719663, ...,  0.01719424,
         -0.02597539,  0.0344    ],
        [ 0.00276902,  0.03371077, -0.01077895, ..., -0.03431912,
          0.04923537, -0.01229758],
        [ 0.0060539 , -0.01094462, -0.00403895, ..., -0.04864769,
          0.0049566 ,  0.01545937],
        ...,
        [-0.02660332,  0.00247737, -0.03074862, ..., -0.03747896,
         -0.039895  , -0.03056909],
        [-0.02660332,  0.00247737, -0.03074862, ..., -0.03747896,
         -0.039895  , -0.03056909],
        [-0.02660332,  0.00247737, -0.03074862, ..., -0.03747896,
         -0.039895  , -0.03056909]]], dtype=float32)>

In [26]:
sample_embed[0][0],sample_embed[0][0].shape,random_sentence[0]

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-0.03648127,  0.03756154,  0.00719663,  0.02718171, -0.01667589,
         0.0119449 , -0.03572067,  0.04780691, -0.0247479 , -0.0271726 ,
         0.04437866, -0.01675339, -0.0161104 ,  0.04161186,  0.04338516,
         0.03963054, -0.02238414,  0.03259781, -0.0188993 ,  0.02638957,
        -0.0414234 ,  0.02022603,  0.01373098, -0.02519732,  0.02890748,
        -0.02852838, -0.04196725, -0.01879833,  0.01467452,  0.00698673,
         0.0264663 ,  0.02086269,  0.00676928, -0.02123946, -0.03664582,
        -0.00973237,  0.02189494,  0.01599075, -0.04925   , -0.00710751,
         0.04974394,  0.02398856, -0.03801499, -0.01954058, -0.00854834,
        -0.0209186 ,  0.01125481, -0.03085341,  0.01986328, -0.00029039,
         0.00618734, -0.00327486, -0.03542854, -0.04632562,  0.02941478,
         0.04521123,  0.03537765,  0.03504339, -0.04355003,  0.03232707,
        -0.00050851, -0.03936306,  0.02914909, -0.02293801,  0.04427956,
  

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline


In [28]:
model0=Pipeline([
("tfidf",TfidfVectorizer()),#convert words to numbers using tfdif
("clf",MultinomialNB())#model the text
])
#fit pipeline to train data
model0.fit(train_sentences,train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [29]:
baseline_score=model0.score(val_sentences,val_labels)# evaluate 
baseline_score # accuracy 

0.7926509186351706

In [30]:
baseline_pred=model0.predict(val_sentences)
baseline_pred[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

#creating function to track and evaluate model 

In [31]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
def calculate_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1 score of a binary classification model.

  Args:
  ----
  y_true = true labels in the form of a 1D array
  y_pred = predicted label in the form of a 1D array

  Returns a dictionary of accuracy, precision, recall and f1-score between y_true and y_pred.
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100 # get accuracy score in percentage value
  # Calculate model precision, recall and f1 score using "weighted" avergage
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  # Create a dictionary of model results
  model_results = {"accuracy": model_accuracy,
                   "precision": model_precision,
                   "recall": model_recall,
                   "f1": model_f1}
  return model_results

In [32]:
baseline_result=calculate_results(y_true=val_labels,y_pred=baseline_pred)
baseline_result

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

#model 1 (Dense model)

In [33]:
from helper_functions import create_tensorboard_callback


In [34]:
SAVE_DIR = "model_logs"


In [35]:
from tensorflow.keras import layers
inputs=(layers.Input(shape=(1,),dtype=tf.string))
x=text_vectorizer(inputs)
x=embedding(x)
x=layers.GlobalAveragePooling1D()(x)
#x=layers.GlobalMaxPool1D()(x)
outputs=layers.Dense(1,activation="sigmoid")(x)
model1=tf.keras.Model(inputs,outputs,name="model1")

In [36]:
model1.summary()

Model: "model1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_2 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
Non-trai

In [37]:
model1.summary()

Model: "model1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_2 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
Non-trai

In [38]:
#compile model
model1.compile(loss=tf.keras.losses.binary_crossentropy,optimizer=tf.keras.optimizers.Adam(),metrics=["accuracy"])


In [39]:
#fit the model
model1_history=model1.fit(x=train_sentences,y=train_labels,epochs=5,validation_data=(val_sentences,val_labels),callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR, 
                                                                     experiment_name="simple_dense_model")])

Saving TensorBoard log files to: model_logs/simple_dense_model/20220712-152007
Epoch 1/5
215/215 [==============================] - 9s 9ms/step - loss: 0.6120 - accuracy: 0.6894 - val_loss: 0.5359 - val_accuracy: 0.7612
Epoch 2/5
215/215 [==============================] - 2s 8ms/step - loss: 0.4416 - accuracy: 0.8205 - val_loss: 0.4724 - val_accuracy: 0.7795
Epoch 3/5
215/215 [==============================] - 2s 7ms/step - loss: 0.3472 - accuracy: 0.8624 - val_loss: 0.4623 - val_accuracy: 0.7874
Epoch 4/5
215/215 [==============================] - 1s 6ms/step - loss: 0.2836 - accuracy: 0.8914 - val_loss: 0.4661 - val_accuracy: 0.7887
Epoch 5/5
215/215 [==============================] - 1s 7ms/step - loss: 0.2368 - accuracy: 0.9137 - val_loss: 0.4792 - val_accuracy: 0.7874


In [40]:
model1.evaluate(val_sentences,val_labels)

24/24 [==============================] - 0s 6ms/step - loss: 0.4792 - accuracy: 0.7874


[0.4792076349258423, 0.787401556968689]

In [41]:
model1_pred_probs=model1.predict(val_sentences)
model1_pred_probs.shape

(762, 1)

In [42]:
model1_pred_probs[0]

array([0.36813033], dtype=float32)

In [43]:
#Convert model prediction probabilites to label format 
model1_preds=tf.squeeze(tf.round(model1_pred_probs))
model1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1.], dtype=float32)>

In [44]:
model1_result=calculate_results(y_true=val_labels,y_pred=model1_preds)
model1_result


{'accuracy': 78.74015748031496,
 'f1': 0.7845557264291478,
 'precision': 0.791897156051696,
 'recall': 0.7874015748031497}

In [45]:
baseline_result

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

#visualazing learned embeddings

In [46]:
#get the vocabulary from  the vectorization layer
words_in_vocab=text_vectorizer.get_vocabulary()
len(words_in_vocab),words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [47]:
model1.summary()

Model: "model1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_2 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
Non-trai

In [48]:
embed_weights=model1.get_layer("embedding").get_weights()[0]
embed_weights.shape

(10000, 128)

In [49]:
import io

# Code to save trained embeddings to file - we got this from here: https://www.tensorflow.org/tutorials/text/word_embeddings#retrieve_the_trained_word_embeddings_and_save_them_to_disk
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [50]:
# Let's download the saved embeddings locally

try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#recurrent neural Networks (RNN)

#model 2 LSTM 
long short term momory 

In [51]:
from tensorflow.keras import layers
inputs=layers.Input(shape=(1,),dtype="string")
x=text_vectorizer(inputs)
x=embedding(x)
#x=layers.LSTM(64,return_sequences=True)(x) # return vector for each word in the Tweet (you can stack RNN cells as long as return_sequences=True)
x = layers.LSTM(64)(x) 
x=layers.Dense(64,activation="relu")(x)
outputs=layers.Dense(1,activation="sigmoid")(x)
model2=tf.keras.Model(inputs,outputs,name="model2")

In [52]:
model2.summary()

Model: "model2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_2 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                            

In [53]:
#compile the model
model2.compile(loss=tf.keras.losses.binary_crossentropy,optimizer=tf.keras.optimizers.Adam(),metrics=["accuracy"])

In [54]:
#fit the model
model2_history=model2.fit(train_sentences,train_labels,epochs=5,validation_data=(val_sentences,val_labels), callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                               experiment_name="model_2_LSTM")])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20220712-152033
Epoch 1/5
215/215 [==============================] - 8s 9ms/step - loss: 0.2157 - accuracy: 0.9269 - val_loss: 0.5748 - val_accuracy: 0.7769
Epoch 2/5
215/215 [==============================] - 1s 7ms/step - loss: 0.1520 - accuracy: 0.9418 - val_loss: 0.6360 - val_accuracy: 0.7861
Epoch 3/5
215/215 [==============================] - 1s 6ms/step - loss: 0.1260 - accuracy: 0.9543 - val_loss: 0.7427 - val_accuracy: 0.7782
Epoch 4/5
215/215 [==============================] - 1s 6ms/step - loss: 0.1018 - accuracy: 0.9612 - val_loss: 0.8428 - val_accuracy: 0.7743
Epoch 5/5
215/215 [==============================] - 1s 6ms/step - loss: 0.0831 - accuracy: 0.9657 - val_loss: 0.7980 - val_accuracy: 0.7756


In [55]:
#make prediction 
model2_pred_probs=model2.predict(val_sentences)
model2_pred_probs[:10]

array([[0.1679305 ],
       [0.9337247 ],
       [0.9999784 ],
       [0.12658238],
       [0.00173642],
       [0.9995147 ],
       [0.7516641 ],
       [0.9999883 ],
       [0.99995995],
       [0.6165463 ]], dtype=float32)

In [56]:
# Round out pred probs and reduce to 1-dimensional array

model_2_preds = tf.squeeze(tf.round(model2_pred_probs))


In [57]:
# Calculate our LSTM model's results

model_2_preds = tf.squeeze(tf.round(model_2_preds))


In [58]:
# Calculate our model 2 results
model_2_results = calculate_results(y_true=val_labels,
                                    y_pred=model_2_preds)
model_2_results

{'accuracy': 77.55905511811024,
 'f1': 0.7746985796594229,
 'precision': 0.7754996204788492,
 'recall': 0.7755905511811023}

#model 3 - GRU model

In [59]:
#build RNN model using GRU cell
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
#x=layers.LSTM(64,return_sequences=True)(x)
#x=layers.GRU(64,)(x)
#x=layers.Dense(64,activation="relu")(x)
x=layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model3 = tf.keras.Model(inputs, outputs, name="model_3_GRU")


In [60]:
#model 3 summary with out GlobalAverage pooling
model3.summary()

Model: "model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_2 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d_1   (None, 128)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
Non

In [61]:
#model 3 summary with our GlobalAverage pooling
model3.summary()

Model: "model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_2 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d_1   (None, 128)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
Non

In [62]:
#compile the model
model3.compile(loss=tf.keras.losses.binary_crossentropy,optimizer=tf.keras.optimizers.Adam(),metrics=["accuracy"])

In [63]:
#fit the model
model3.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                     experiment_name="model_3_GRU")])

Saving TensorBoard log files to: model_logs/model_3_GRU/20220712-152057
Epoch 1/5
215/215 [==============================] - 2s 5ms/step - loss: 0.5097 - accuracy: 0.8456 - val_loss: 0.4984 - val_accuracy: 0.7782
Epoch 2/5
215/215 [==============================] - 1s 4ms/step - loss: 0.3154 - accuracy: 0.9040 - val_loss: 0.4585 - val_accuracy: 0.7953
Epoch 3/5
215/215 [==============================] - 1s 4ms/step - loss: 0.2382 - accuracy: 0.9241 - val_loss: 0.4664 - val_accuracy: 0.7927
Epoch 4/5
215/215 [==============================] - 1s 4ms/step - loss: 0.1953 - accuracy: 0.9391 - val_loss: 0.4837 - val_accuracy: 0.7848
Epoch 5/5
215/215 [==============================] - 1s 4ms/step - loss: 0.1656 - accuracy: 0.9499 - val_loss: 0.5125 - val_accuracy: 0.7808


In [64]:
model3_pred_probs=model3.predict(val_sentences)
model3_pred_probs[:10]

array([[0.32866985],
       [0.7508717 ],
       [0.99756753],
       [0.12010258],
       [0.03653713],
       [0.9436027 ],
       [0.8601472 ],
       [0.9973852 ],
       [0.98801917],
       [0.37632224]], dtype=float32)

In [65]:
model3_preds=tf.squeeze(tf.round(model3_pred_probs))
model3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [66]:
model3_result=calculate_results(y_true=val_labels,y_pred=model3_preds)
model3_result

{'accuracy': 78.08398950131233,
 'f1': 0.7782630349987989,
 'precision': 0.7841274438015813,
 'recall': 0.7808398950131233}

#Model4 bidirectional RNN
normal RNN's go from left to right 
but bidirectional RNN goes from right to left as well as left to right

In [67]:
#build a model with tensorflow 
from tensorflow.keras import layers
inputs=layers.Input(shape=(1,),dtype="string")
x=text_vectorizer(inputs)
x=embedding(x)
x=layers.Bidirectional(layers.LSTM(64))(x)
#x=layers.Bidirectional(layers.GRU(64,return_sequences=True))(x)
outputs=layers.Dense(1,activation="sigmoid")(x)
model4=tf.keras.Model(inputs,outputs,name="model4_bidirectional")

                                                             

In [68]:
#our model summary
model4.summary()
#the shape is 128 (instead of 64 ) because we are going in 2direcitons (so it dobules the value )

Model: "model4_bidirectional"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_2 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 bidirectional (Bidirectiona  (None, 128)              98816     
 l)                                                              
                                                                 
 dense_4 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,378,945
Trainable params: 1,37

In [69]:
#compile the model
model4.compile(loss=tf.keras.losses.binary_crossentropy,optimizer=tf.keras.optimizers.Adam(),metrics=["accuracy"])

In [70]:
# fit the model
model4.history=model4.fit(train_sentences,train_labels,epochs=5,validation_data=(val_sentences,val_labels),
                          callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                     experiment_name="model_4")])

Saving TensorBoard log files to: model_logs/model_4/20220712-152104
Epoch 1/5
215/215 [==============================] - 5s 11ms/step - loss: 0.1329 - accuracy: 0.9571 - val_loss: 0.8142 - val_accuracy: 0.7730
Epoch 2/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0832 - accuracy: 0.9669 - val_loss: 0.9534 - val_accuracy: 0.7664
Epoch 3/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0627 - accuracy: 0.9727 - val_loss: 1.2031 - val_accuracy: 0.7756
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0520 - accuracy: 0.9762 - val_loss: 1.3083 - val_accuracy: 0.7690
Epoch 5/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0522 - accuracy: 0.9740 - val_loss: 1.2528 - val_accuracy: 0.7664


In [71]:
#make predictions 
model4_pred_probs=model4.predict(val_sentences)
model4_pred_probs[:10]

array([[1.1349627e-02],
       [4.6406773e-01],
       [9.9988353e-01],
       [3.2416698e-02],
       [2.5714104e-05],
       [9.9927109e-01],
       [9.4734281e-01],
       [9.9996817e-01],
       [9.9989784e-01],
       [9.6923190e-01]], dtype=float32)

In [72]:
#convert pred probs to pred labels
model4_preds=tf.squeeze(tf.round(model4_pred_probs))
model4_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 0., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [73]:
#calculate the result of our bidirectional model
model4_result=calculate_results(y_true=val_labels,y_pred=model4_preds)
model4_result

{'accuracy': 76.64041994750657,
 'f1': 0.7640154753782149,
 'precision': 0.7684333697179313,
 'recall': 0.7664041994750657}

In [74]:
model3_result

{'accuracy': 78.08398950131233,
 'f1': 0.7782630349987989,
 'precision': 0.7841274438015813,
 'recall': 0.7808398950131233}

it seems that bidirectional model performed worse than unidirectional 

#CNN for text  ( CONV1D)

In [75]:
#see how CONV1D works
embedding_test=embedding(text_vectorizer(["this is a test sentence "]))
conv1d=layers.Conv1D(filters=32,kernel_size=5,activation="relu",padding="valid")
conv1d_output=conv1d(embedding_test)
max_pool=layers.GlobalMaxPooling1D()# get the most important features
max_pool_out=max_pool(conv1d_output)
embedding_test.shape,conv1d_output.shape,max_pool_out.shape

 # kernel size = setting this to 5 means it'll look at 5 words at a time, 3 would mean 3 words at a time

(TensorShape([1, 15, 128]), TensorShape([1, 11, 32]), TensorShape([1, 32]))

In [76]:
#create CNN model with functional API

inputs=layers.Input(shape=(1,),dtype=tf.string)
x=text_vectorizer(inputs)
x=embedding(x)
x=layers.Conv1D(64,kernel_size=5,activation="relu",padding="valid")(x)
x=layers.GlobalMaxPooling1D()(x)
#x=layers.Conv1D(64,5,activation="relu",padding="valid")(x)
#x=layers.GlobalMaxPooling1D()(x)
#x=layers.Conv1D(64,5,activation="relu",padding="valid")(x)
#x=layers.GlobalMaxPooling1D()(x)
outputs=layers.Dense(1,activation="sigmoid")(x)
model5=tf.keras.Model(inputs,outputs,name="model5_with_CNN")

In [77]:
model5.compile(loss=tf.keras.losses.binary_crossentropy,optimizer=tf.keras.optimizers.Adam(),metrics=["accuracy"])

In [78]:
model5.summary()

Model: "model5_with_CNN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_2 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 conv1d_1 (Conv1D)           (None, 11, 64)            41024     
                                                                 
 global_max_pooling1d_1 (Glo  (None, 64)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_5 (Dense)             (None, 1)             

In [79]:
history5=model5.fit(train_sentences,train_labels,epochs=5,validation_data=(val_sentences,val_labels), callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                     experiment_name="model5")])

Saving TensorBoard log files to: model_logs/model5/20220712-152123
Epoch 1/5
215/215 [==============================] - 4s 9ms/step - loss: 0.1277 - accuracy: 0.9553 - val_loss: 0.8659 - val_accuracy: 0.7795
Epoch 2/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0870 - accuracy: 0.9676 - val_loss: 0.9793 - val_accuracy: 0.7756
Epoch 3/5
215/215 [==============================] - 2s 7ms/step - loss: 0.0692 - accuracy: 0.9747 - val_loss: 1.0807 - val_accuracy: 0.7651
Epoch 4/5
215/215 [==============================] - 1s 5ms/step - loss: 0.0613 - accuracy: 0.9761 - val_loss: 1.1281 - val_accuracy: 0.7690
Epoch 5/5
215/215 [==============================] - 1s 5ms/step - loss: 0.0561 - accuracy: 0.9777 - val_loss: 1.1760 - val_accuracy: 0.7572


In [80]:
# prediction
model5_pred_probs=model5.predict(val_sentences)
model5_pred_probs[:10]

array([[5.1488435e-01],
       [2.3722199e-01],
       [9.9977964e-01],
       [4.4106711e-02],
       [1.6126069e-07],
       [9.9593687e-01],
       [9.9487728e-01],
       [9.9995148e-01],
       [9.9999833e-01],
       [8.2692325e-01]], dtype=float32)

In [81]:
#convert model5 pred probs to labels
model5_preds=tf.squeeze(tf.round(model5_pred_probs))
model5_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([1., 0., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [82]:
model5_result=calculate_results(y_true=val_labels,y_pred=model5_preds)
model5_result ,"model 5 vs baseline model", baseline_result

({'accuracy': 75.7217847769029,
  'f1': 0.755217155129904,
  'precision': 0.7581583214678428,
  'recall': 0.7572178477690289},
 'model 5 vs baseline model',
 {'accuracy': 79.26509186351706,
  'f1': 0.7862189758049549,
  'precision': 0.8111390004213173,
  'recall': 0.7926509186351706})

our CNN model still worse than base model 

#Model6 Pretrained Model ( USE : universal sentence Encoder)

In [83]:
sample_sentence="There's a flood in my street!"

In [84]:
import tensorflow_hub as hub
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4") # load Universal Sentence Encoder
embed_samples = embed([sample_sentence,
                      "When you call the universal sentence encoder on a sentence, it turns it into numbers."])

print(embed_samples[0][:50])

tf.Tensor(
[-0.01157028  0.0248591   0.02878048 -0.012715    0.03971538  0.0882776
  0.02680984  0.05589836 -0.0106873  -0.00597291  0.00639323 -0.01819518
  0.00030813  0.09105888  0.05874644 -0.03180628  0.01512474 -0.05162929
  0.00991367 -0.06865347 -0.04209306  0.02678981  0.03011006  0.00321069
 -0.00337973 -0.04787357  0.0226672  -0.00985925 -0.04063613 -0.01292092
 -0.04666384  0.05630299 -0.03949255  0.00517686  0.02495829 -0.0701444
  0.02871508  0.04947684 -0.00633979 -0.08960192  0.02807118 -0.00808364
 -0.01360602  0.0599865  -0.10361787 -0.05195374  0.00232954 -0.02332531
 -0.03758105  0.03327728], shape=(50,), dtype=float32)


In [85]:
# We can use this encoding layer in place of our text_vectorizer and embedding layer
sentence_encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                        input_shape=[], # shape of inputs coming to our model 
                                        dtype=tf.string, # data type of inputs coming to the USE layer
                                        trainable=False, # keep the pretrained weights 
                                        name="USE") 

In [104]:
#create model with Sequential API
# in this model we dont need text_vectorization
model6=tf.keras.Sequential([
sentence_encoder_layer,
#layers.Dense(64,activation="relu"),
layers.Dense(64,activation="relu"),

layers.Dense(1,activation="relu")                     
],name="model6")


In [105]:
model6.compile(loss=tf.keras.losses.binary_crossentropy,optimizer=tf.keras.optimizers.Adam(),metrics=["accuracy"])

In [106]:
model6.summary()

Model: "model6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_12 (Dense)            (None, 64)                32832     
                                                                 
 dense_13 (Dense)            (None, 1)                 65        
                                                                 
Total params: 256,830,721
Trainable params: 32,897
Non-trainable params: 256,797,824
_________________________________________________________________


In [107]:
# Train a classifier on top of pretrained embeddings
model6_history=model6.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, 
                                                                     "tf_hub_sentence_encoder")])

Saving TensorBoard log files to: model_logs/tf_hub_sentence_encoder/20220712-153201
Epoch 1/5
215/215 [==============================] - 5s 16ms/step - loss: 0.6069 - accuracy: 0.7631 - val_loss: 0.5131 - val_accuracy: 0.7927
Epoch 2/5
215/215 [==============================] - 4s 18ms/step - loss: 0.4526 - accuracy: 0.8105 - val_loss: 0.5434 - val_accuracy: 0.8018
Epoch 3/5
215/215 [==============================] - 3s 12ms/step - loss: 0.4266 - accuracy: 0.8190 - val_loss: 0.4897 - val_accuracy: 0.8097
Epoch 4/5
215/215 [==============================] - 3s 13ms/step - loss: 0.4065 - accuracy: 0.8247 - val_loss: 0.5771 - val_accuracy: 0.8071
Epoch 5/5
215/215 [==============================] - 3s 12ms/step - loss: 0.3820 - accuracy: 0.8254 - val_loss: 0.6053 - val_accuracy: 0.8123


In [97]:
model6_pred_probs=model6.predict(val_sentences)
model6_pred_probs[:10]


array([[0.25676566],
       [0.78913933],
       [1.3651987 ],
       [0.12377359],
       [0.70681196],
       [0.5453131 ],
       [1.2395757 ],
       [1.152045  ],
       [1.0847312 ],
       [0.10033672]], dtype=float32)

In [98]:
model6_preds=tf.squeeze(tf.round(model6_pred_probs))
model6_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 1., 1., 1., 1., 1., 0.], dtype=float32)>

In [99]:
model6_reuslt=calculate_results(val_labels, model6_preds)
model6_reuslt


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'accuracy': 80.5774278215223,
 'f1': 0.80608784644625,
 'precision': 0.8120703117495466,
 'recall': 0.8057742782152231}

#model7 with Smaller dataset(10%)

In [108]:
#this is wrong way to split data 
train_10_percent = train_df_shuffled[["text", "target"]].sample(frac=0.1, random_state=42)
train_sentences_10_percent = train_10_percent["text"].to_list()
train_labels_10_percent = train_10_percent["target"].to_list() 
len(train_sentences_10_percent), len(train_labels_10_percent)

(761, 761)

In [109]:
model_7 = tf.keras.models.clone_model(model6)

# Compile model
model_7.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

# Get a summary (will be same as model_6)
model_7.summary()

Model: "model6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_12 (Dense)            (None, 64)                32832     
                                                                 
 dense_13 (Dense)            (None, 1)                 65        
                                                                 
Total params: 256,830,721
Trainable params: 32,897
Non-trainable params: 256,797,824
_________________________________________________________________


In [110]:
# Fit the model to 10% of the training data
model_7_history = model_7.fit(x=train_sentences_10_percent,
                              y=train_labels_10_percent,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, "10_percent_tf_hub_sentence_encoder")])

Saving TensorBoard log files to: model_logs/10_percent_tf_hub_sentence_encoder/20220712-155946
Epoch 1/5
24/24 [==============================] - 4s 38ms/step - loss: 1.6292 - accuracy: 0.5427 - val_loss: 0.6668 - val_accuracy: 0.5551
Epoch 2/5
24/24 [==============================] - 1s 25ms/step - loss: 0.5957 - accuracy: 0.6623 - val_loss: 0.5398 - val_accuracy: 0.7651
Epoch 3/5
24/24 [==============================] - 1s 24ms/step - loss: 0.5137 - accuracy: 0.7858 - val_loss: 0.4889 - val_accuracy: 0.8005
Epoch 4/5
24/24 [==============================] - 0s 21ms/step - loss: 0.4748 - accuracy: 0.8016 - val_loss: 0.4521 - val_accuracy: 0.8097
Epoch 5/5
24/24 [==============================] - 1s 24ms/step - loss: 0.4423 - accuracy: 0.8029 - val_loss: 0.4216 - val_accuracy: 0.8071
